<a href="https://colab.research.google.com/github/spencer-hann/DeepReinforcementLearning/blob/master/dq_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [2]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1013'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1013'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1


**OpenAI Gym**

We're gonna spend several next weeks learning algorithms that solve decision processes. We are then in need of some interesting decision problems to test our algorithms.

That's where OpenAI gym comes into play. It's a python library that wraps many classical decision problems including robot control, videogames and board games.

So here's how it works:


In [0]:
import gym
from gym import wrappers

env = gym.make('CartPole-v0').unwrapped
env = wrappers.Monitor(env, "./gym-results", force=True)
env.reset()
for _ in range(1000):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    if done: break
env.close()



In [5]:
import io
import base64
from IPython.display import HTML

video = io.open('./gym-results/openaigym.video.%s.video000000.mp4' % env.file_infix, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''
    <video width="720" height="auto" alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''
.format(encoded.decode('ascii')))


reset() - reset environment to initial state, return first observation

render() - show current environment state (a more colorful version :) )

step(a) - commit action a and return (new observation, reward, is done, info)


In [0]:
import gym
from gym import wrappers
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
from math import exp

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

env = gym.make('CartPole-v1')
env = wrappers.Monitor(env, "./gym-results", force=True).unwrapped
env.reset()

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [0]:
class DQN(nn.Module):

    def __init__(self, h, w, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)

        # Number of Linear input connections depends on output of conv2d layers
        # and therefore the input image size, so compute it.
        def conv2d_size_out(size, kernel_size = 5, stride = 2):
            return (size - (kernel_size - 1) - 1) // stride  + 1
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * 32
        self.head = nn.Linear(linear_input_size, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))

In [9]:
resize = T.Compose([T.ToPILImage(),
                    T.Resize((400,40), interpolation=Image.CUBIC),
                    T.ToTensor()])


def get_cart_location(screen_width):
    world_width = env.x_threshold * 2
    scale = screen_width / world_width
    return int(env.state[0] * scale + screen_width / 2.0)  # MIDDLE OF CART

def f(x):
    if x == 255:
        return 1
    return 0
f = np.vectorize(f)

def get_screen():
    # Returned screen requested by gym is 400x600x3, but is sometimes larger
    # such as 800x1200x3. Transpose it into torch order (CHW).
    screen = env.render(mode='rgb_array').transpose((2, 0, 1))
    screen = screen.min(axis=0)
#     screen = f(screen)
    # Cart is in the lower half, so strip off the top and bottom of the screen
    screen_height, screen_width = screen.shape
    screen = screen[:, int(screen_height*0.4):int(screen_height * 0.8)]
    view_width = int(screen_width * 0.6)
    cart_location = get_cart_location(screen_width)
    if cart_location < view_width // 2:
        slice_range = slice(view_width)
    elif cart_location > (screen_width - view_width // 2):
        slice_range = slice(-view_width, None)
    else:
        slice_range = slice(cart_location - view_width // 2,
                            cart_location + view_width // 2)
    # Strip off the edges, so that we have a square image centered on a cart
    screen = screen[:, slice_range]
    # Convert to float, rescale, convert to torch tensor
    # (this doesn't require a copy)
    screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
    screen = torch.from_numpy(screen)
    # Resize, and add a batch dimension (BCHW)
    screen = resize(screen).unsqueeze(0).to(device)
    return screen

env.reset()
plt.figure()
img = get_screen().cpu().squeeze(0).squeeze(0).numpy()
plt.imshow(img, interpolation='none', cmap="gray")
plt.title('Example extracted screen')
plt.show()

In [0]:
BATCH_SIZE = 32
GAMMA = 0.999
EPS_START = .999
EPS_END = 0.1
EPS_DECAY = 4000
TARGET_UPDATE = 10

# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
init_screen = get_screen()
_, _, screen_height, screen_width = init_screen.shape

# Get number of actions from gym action space
n_actions = env.action_space.n

policy_net = DQN(screen_height, screen_width, n_actions).to(device)
target_net = DQN(screen_height, screen_width, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters(),lr=0.0001)#, momentum=0.4)
memory = ReplayMemory(10_000)


steps_done = 0

def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)


episode_durations = []


def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())
    plt.grid()

#     plt.pause(0.001)  # pause a bit so that plots are updated
    #if is_ipython:
        #display.clear_output(wait=True)
        #display.display(plt.gcf())

In [0]:
thresholds = []
episode_durations = []

def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy(), label="durations")
    plt.plot(np.array(thresholds)*100, label="epsilon")
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy(), label="mean")
    plt.grid(); plt.legend()

#     plt.pause(0.001)  # pause a bit so that plots are updated
    #if is_ipython:
        #display.clear_output(wait=True)
        #display.display(plt.gcf())

In [0]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.uint8)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [13]:
from tqdm import tqdm

num_episodes = 2000
for i_episode in tqdm(range(num_episodes)):
    # Initialize the environment and state
    
    if i_episode > num_episodes * .5:
        EPS_END = 0    
    
    env.reset()
    last_screen = get_screen()
    current_screen = get_screen()
    state = current_screen - last_screen
    for t in count():
        # Select and perform an action
        action = select_action(state)
        _, reward, done, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)

        # Observe new state
        last_screen = current_screen
        current_screen = get_screen()
        if not done:
            next_state = current_screen - last_screen
        else:
            next_state = None

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        optimize_model()
        if done or t > 400:
            episode_durations.append(t + 1)
            eps_threshold = EPS_END + (EPS_START - EPS_END) * \
                exp(-1. * steps_done / EPS_DECAY)
            thresholds.append(eps_threshold)
            #plot_durations()
            break
    if (i_episode+1) % 50 == 0:
        episode_durations_mean = np.mean(episode_durations[-100:])
        print(' Mean last 50:', episode_durations_mean,'\t',np.mean(thresholds[-50:]))
        if episode_durations_mean > 100:
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')

  3%|▎         | 51/2000 [00:14<06:43,  4.83it/s]

 Mean last 50: 25.28 	 0.8683571608745566


  5%|▌         | 100/2000 [00:30<07:25,  4.26it/s]

 Mean last 50: 26.06 	 0.6539747514420076


  8%|▊         | 151/2000 [00:48<08:56,  3.45it/s]

 Mean last 50: 28.34 	 0.4918769908076684


 10%|█         | 200/2000 [01:08<16:04,  1.87it/s]

 Mean last 50: 31.75 	 0.3685104573816315


 12%|█▎        | 250/2000 [01:32<12:12,  2.39it/s]

 Mean last 50: 36.98 	 0.26558714072245865


 15%|█▌        | 300/2000 [01:58<17:52,  1.59it/s]

 Mean last 50: 42.53 	 0.19905772405610128


 18%|█▊        | 350/2000 [02:24<15:16,  1.80it/s]

 Mean last 50: 43.47 	 0.1570858475243601


 20%|██        | 400/2000 [02:52<13:01,  2.05it/s]

 Mean last 50: 45.19 	 0.13279543859892953


 22%|██▎       | 450/2000 [03:16<08:30,  3.04it/s]

 Mean last 50: 44.14 	 0.11840735628776619


 25%|██▌       | 500/2000 [03:58<20:26,  1.22it/s]

 Mean last 50: 55.98 	 0.1093037056182173


 28%|██▊       | 550/2000 [04:36<16:37,  1.45it/s]

 Mean last 50: 68.06 	 0.10392255670325319


 30%|███       | 600/2000 [05:18<24:54,  1.07s/it]

 Mean last 50: 68.03 	 0.10172049987257413


 32%|███▎      | 650/2000 [05:54<13:11,  1.71it/s]

 Mean last 50: 66.75 	 0.1007471367626923


 35%|███▌      | 700/2000 [06:29<12:41,  1.71it/s]

 Mean last 50: 60.39 	 0.1003441425427794


 38%|███▊      | 750/2000 [07:08<12:03,  1.73it/s]

 Mean last 50: 62.09 	 0.10015898508965618


 40%|████      | 800/2000 [07:47<13:31,  1.48it/s]

 Mean last 50: 66.22 	 0.10007068863316002


 42%|████▎     | 850/2000 [08:15<10:47,  1.78it/s]

 Mean last 50: 57.31 	 0.10003338830292051


 45%|████▌     | 900/2000 [08:58<18:53,  1.03s/it]

 Mean last 50: 59.7 	 0.10001700438358643


 48%|████▊     | 950/2000 [09:34<15:45,  1.11it/s]

 Mean last 50: 67.11 	 0.10000704951141896


 50%|█████     | 1000/2000 [10:04<14:05,  1.18it/s]

 Mean last 50: 55.92 	 0.10000338693521296


 52%|█████▎    | 1050/2000 [10:44<14:19,  1.11it/s]

 Mean last 50: 59.26 	 0.0020018467489261605


 55%|█████▌    | 1100/2000 [11:24<18:37,  1.24s/it]

 Mean last 50: 67.98 	 8.312100784274712e-07


 57%|█████▊    | 1150/2000 [12:02<11:15,  1.26it/s]

 Mean last 50: 66.11 	 3.533057248648389e-07


 60%|██████    | 1200/2000 [12:38<11:11,  1.19it/s]

 Mean last 50: 62.37 	 1.6169548618977742e-07


 62%|██████▎   | 1250/2000 [13:17<06:08,  2.04it/s]

 Mean last 50: 62.39 	 6.975620231670948e-08


 65%|██████▌   | 1300/2000 [13:49<06:03,  1.92it/s]

 Mean last 50: 59.85 	 3.258328041385152e-08


 68%|██████▊   | 1350/2000 [14:23<09:46,  1.11it/s]

 Mean last 50: 56.19 	 1.7384079864265092e-08


 70%|███████   | 1400/2000 [15:09<09:14,  1.08it/s]

 Mean last 50: 68.29 	 7.211611685088087e-09


 72%|███████▎  | 1450/2000 [15:46<05:37,  1.63it/s]

 Mean last 50: 70.12 	 2.918536474460138e-09


 75%|███████▌  | 1500/2000 [16:31<12:13,  1.47s/it]

 Mean last 50: 69.17 	 1.3472262932093273e-09


 78%|███████▊  | 1550/2000 [17:25<08:00,  1.07s/it]

 Mean last 50: 85.15 	 4.6560541712247783e-10


 80%|████████  | 1601/2000 [18:10<02:44,  2.42it/s]

 Mean last 50: 84.2 	 1.4465014508660132e-10


 83%|████████▎ | 1651/2000 [19:07<04:47,  1.22it/s]

 Mean last 50: 86.43 	 5.3384419290267434e-11


 85%|████████▌ | 1700/2000 [19:47<05:19,  1.07s/it]

 Mean last 50: 82.97 	 1.9073550813118806e-11


 88%|████████▊ | 1750/2000 [20:38<03:57,  1.05it/s]

 Mean last 50: 78.4 	 7.21474217143462e-12


 90%|█████████ | 1800/2000 [21:26<03:35,  1.08s/it]

 Mean last 50: 84.73 	 2.429065939468491e-12


 92%|█████████▎| 1850/2000 [22:13<01:46,  1.41it/s]

 Mean last 50: 81.45 	 8.59011222883451e-13


 95%|█████████▌| 1900/2000 [23:00<01:19,  1.26it/s]

 Mean last 50: 80.69 	 3.068884080590233e-13


 98%|█████████▊| 1950/2000 [23:37<00:33,  1.50it/s]

 Mean last 50: 71.08 	 1.2859895325697335e-13


100%|██████████| 2000/2000 [24:14<00:00,  1.36it/s]

 Mean last 50: 62.72 	 5.847165980662112e-14
Complete


env.length= 0.5
mean: 62.72
max: 311


In [14]:
env.render()
env.close()
plt.ioff()
plot_durations()
plt.show()
print('env.length=',env.length)
print('mean:',np.mean(episode_durations[-100:]))
print('max:', max(episode_durations))

In [15]:
policy_net.eval()

DQN(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(2, 2))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 32, kernel_size=(5, 5), stride=(2, 2))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (head): Linear(in_features=3008, out_features=2, bias=True)
)

In [16]:
from tqdm import tqdm

thresholds = []
episode_durations = []

env.length = .6

num_episodes = 400
for i_episode in tqdm(range(num_episodes)):
    # Initialize the environment and state
    
    env.reset()
    last_screen = get_screen()
    current_screen = get_screen()
    state = current_screen - last_screen
    for t in count():
        # Select and perform an action
        action = select_action(state)
        _, reward, done, _ = env.step(action.item())
        #reward = torch.tensor([reward], device=device)

        # Observe new state
        last_screen = current_screen
        current_screen = get_screen()
        if not done:
            next_state = current_screen - last_screen
        else:
            next_state = None

        # Store the transition in memory
        #memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        #optimize_model()
        if done or t > 400:
            episode_durations.append(t + 1)
            eps_threshold = EPS_END + (EPS_START - EPS_END) * \
                exp(-1. * steps_done / EPS_DECAY)
            thresholds.append(eps_threshold)
            #plot_durations()
            break
    if (i_episode+1) % 50 == 0:
        print(' Mean last 50:', np.mean(episode_durations[-100:]),'\t',np.mean(thresholds[-50:]))
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')

 12%|█▎        | 50/400 [00:28<04:27,  1.31it/s]

 Mean last 50: 93.12 	 2.3486442135577948e-14


 25%|██▌       | 100/400 [00:56<02:13,  2.25it/s]

 Mean last 50: 92.39 	 7.0620096362695626e-15


 38%|███▊      | 150/400 [01:24<02:16,  1.84it/s]

 Mean last 50: 94.11 	 2.2136029241249866e-15


 50%|█████     | 200/400 [01:50<01:26,  2.31it/s]

 Mean last 50: 90.74 	 7.181663781873876e-16


 62%|██████▎   | 250/400 [02:18<01:35,  1.58it/s]

 Mean last 50: 89.59 	 2.44072398737314e-16


 75%|███████▌  | 300/400 [02:46<01:08,  1.46it/s]

 Mean last 50: 93.54 	 7.560846566899829e-17


 88%|████████▊ | 350/400 [03:14<00:29,  1.68it/s]

 Mean last 50: 93.29 	 2.2581253952024407e-17


100%|██████████| 400/400 [03:37<00:00,  2.25it/s]

 Mean last 50: 84.1 	 7.556443159907797e-18
Complete


env.length= 0.6
mean: 84.1
max: 302


In [17]:
env.render()
env.close()
plt.ioff()
plot_durations()
plt.show()
print('env.length=',env.length)
print('mean:',np.mean(episode_durations[-100:]))
print('max:', max(episode_durations))

In [18]:
from tqdm import tqdm

thresholds = []
episode_durations = []

env.length = .4

num_episodes = 400
for i_episode in tqdm(range(num_episodes)):
    # Initialize the environment and state
    
    env.reset()
    last_screen = get_screen()
    current_screen = get_screen()
    state = current_screen - last_screen
    for t in count():
        # Select and perform an action
        action = select_action(state)
        _, reward, done, _ = env.step(action.item())
        #reward = torch.tensor([reward], device=device)

        # Observe new state
        last_screen = current_screen
        current_screen = get_screen()
        if not done:
            next_state = current_screen - last_screen
        else:
            next_state = None

        # Store the transition in memory
        #memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        #optimize_model()
        if done or t > 400:
            episode_durations.append(t + 1)
            eps_threshold = EPS_END + (EPS_START - EPS_END) * \
                exp(-1. * steps_done / EPS_DECAY)
            thresholds.append(eps_threshold)
            #plot_durations()
            break
    if (i_episode+1) % 50 == 0:
        print(' Mean last 50:', np.mean(episode_durations[-100:]),'\t',np.mean(thresholds[-50:]))
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')

 13%|█▎        | 51/400 [00:14<01:28,  3.93it/s]

 Mean last 50: 44.1 	 3.7205330335444885e-18


 25%|██▌       | 100/400 [00:27<01:35,  3.14it/s]

 Mean last 50: 44.64 	 2.1005679224437703e-18


 38%|███▊      | 151/400 [00:42<00:46,  5.38it/s]

 Mean last 50: 46.77 	 1.1522652005540529e-18


 50%|████▉     | 199/400 [00:58<01:31,  2.20it/s]

 Mean last 50: 49.69 	 6.296949378460972e-19


 62%|██████▎   | 250/400 [01:13<00:44,  3.39it/s]

 Mean last 50: 49.61 	 3.360989401538305e-19


 75%|███████▌  | 300/400 [01:27<00:32,  3.10it/s]

 Mean last 50: 46.09 	 1.8804169182203e-19


 88%|████████▊ | 350/400 [01:42<00:13,  3.59it/s]

 Mean last 50: 46.68 	 1.0605603561662824e-19


100%|██████████| 400/400 [01:56<00:00,  4.20it/s]

 Mean last 50: 48.15 	 5.675142684617052e-20
Complete


env.length= 0.4
mean: 48.15
max: 163


In [19]:
env.render()
env.close()
plt.ioff()
plot_durations()
plt.show()
print('env.length=',env.length)
print('mean:',np.mean(episode_durations[-100:]))
print('max:', max(episode_durations))